In [1]:
import numpy as np
np.random.seed(0)

from scipy.io import loadmat
from scipy import optimize

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import NonUniformImage
from matplotlib import cm
matplotlib.style.use('ggplot')
%matplotlib inline

%load_ext autoreload
%autoreload 2

# 0 Data Structure

In [2]:
pathToDataFile = 'ex4data1.mat'
data = loadmat(pathToDataFile)
pathToWeightsFile = 'ex4weights.mat'
weights = loadmat(pathToWeightsFile)
print(weights['Theta1'].shape)
print(weights['Theta2'].shape)
print(type(weights))

(25, 401)
(10, 26)
<class 'dict'>


# 1 Nural Network
## 1.1 Forward Porpagation
<img src="forwardPropagation.png">

In [3]:
def getData(pathToDataFile):
    data = loadmat(pathToDataFile)
    X = data['X']
    y = data['y']
    return X, y

def generateBeta(layer):
    '''Generate beta-matrix for every layer in Neural Network'''
    betaSet = ()
    for i in range(len(layer)-1):
#         recommendation from Andrew Ng window is ±(6/(inLayer + outLayer))**0.5
        low, high = -(6/(layer[i]+layer[i+1]))**0.5, (6/(layer[i]+layer[i+1]))**0.
        betaSet += (np.random.uniform(low,high,(layer[i+1], layer[i]+1)),)
#         betaSet += (np.zeros((outLayer, inLayer+1)),)
    return betaSet

def flattenBeta(betaSet):
    flatBeta = betaSet[0].flatten()
    for beta in betaSet[1:]:
        flatBeta = np.concatenate((flatBeta, beta.flatten()), axis=-1)
    return flatBeta

def reshapeBeta(Beta, layer):
    splitIndex = 0
    splitIndices = []
    for i in range(len(layer)-2):
        splitIndex += (layer[i]+1)*layer[i+1]
        splitIndices += [splitIndex]
    splitBeta = np.split(Beta, splitIndices)
    reshapedBeta = ()
    for i in range(len(splitBeta)):
        reshapedBeta += (splitBeta[i].reshape(layer[i+1],layer[i]+1),)
    return reshapedBeta
    
def sigmoid(z):
    return 1/(1+np.exp(-z))

def forwardPropagation(flatBeta, layer, flatX, sampleSize):
    '''Forward Propagation is the hypothesis function for Neural Networks'''
    betaSet = reshapeBeta(flatBeta, layer)
#     H_0 (5000, 400)
    H = flatX.reshape(sampleSize, -1)
#     Z_H = ()
    H_byLayer = ()
    for beta in betaSet:
#         print(H.shape)
#         Z_l (5000, k_l); l is the number of layers [0, ...,l]; k is the number of neurons in a layer l [1,...,k]
        Z = np.dot(np.insert(H, 0, 1, axis=1), beta.T)
#         H_l (5000, k_l); l is the number of layers [0, ...,l]; k is the number of neurons in a layer l [1,...,k]
        H = sigmoid(Z)
#         Z_H += ((Z, H),)
        H_byLayer += (H,)
#     H_2 (5000, 10)
    return H_byLayer

def sigmoidGradient(Z):
    return sigmoid(Z)*(1-sigmoid(Z))

def costFunction(flatBeta, layer, flatX, sampleSize, y, yUnique, iLambda = 0.):
    X = flatX.reshape(sampleSize, -1)
    Y = np.array([yUnique]* y.shape[0]) == y
    betaSet = reshapeBeta(flatBeta, layer)
    J = 0
    for n in range(sampleSize):
        x_n = X[n:n+1,:]
        y_n = Y[n:n+1,:]
#         hypothesis vector h_n(1, 10)
        h_n = forwardPropagation(flatBeta, layer, x_n, 1)[len(betaSet)-1]
#         cost function scalar j_n(1, 1) = y_n(1, 10)*h_n.T(10, 1)
        j_n = (- np.dot(y_n, np.log(h_n).T) - np.dot((1-y_n), np.log(1-h_n).T))
        J += j_n
#     regularisation term (R)
    cummulativeR = 0
    for beta in betaSet:
        cummulativeR += np.sum(beta*beta) #element-wise multiplication
    cummulativeR *= iLambda/(2*sampleSize)
    return J[0][0]/sampleSize + cummulativeR

### 1.1.1 Neural Network Initialisation

The input-data matrix X(5000, 400) is comprised of 5000 digit images 20 by 20 pixels (400 pixels).<br>
The output-data vector Y(5000,1) is comprised of 5000 assigned digits (1 through 10; 10 represents figure '0').<br>
The neural network in this work has 1 input layer (400 neurons), one hidden layer (25 neurons) and an output layer (10 neurons).
To initialise a simple neural network, one has to do the following:
1. set the number of neurons in every layer (including input and output layers)
2. extract and flatten input matrix X
3. transform output Y
3. initialise Beat matrix

In [4]:
# Set number of neurons in every layer (including input and output layers)
layer = 400, 25, 10
# Extract and flatten input matrix X
X, y = getData(pathToDataFile)
sampleSize, numVariables = X.shape
flatX = X.flatten()
yUnique = np.unique(y)
# Initialise Beat matrix
betaTest = flattenBeta((weights['Theta1'], weights['Theta2']))
betaInitial = flattenBeta(generateBeta(layer))
print(X.shape)
print(y.shape)
for beta in generateBeta(layer): print(beta.shape)

(5000, 400)
(5000, 1)
(25, 401)
(10, 26)


## 1.1.2 Forward-Propagation Test

In [5]:
# either transformed Y or y together with yUnique can be suplied to a function
# Y = np.array([yUnique]* y.shape[0]) == y
# print(Y[0:0+1,:].shape)

In [6]:
print(forwardPropagation(betaTest, layer, flatX, sampleSize)[1].shape)
print(forwardPropagation(betaTest, layer, X[0:0+1,:], 1)[1].shape)

(5000, 10)
(1, 10)


In [7]:
print(X.shape)
print(X[0][None,:].shape)
# costFunction(betaTest, layer, X.flatten(), sampleSize, y, yUnique, iLambda = 0.)
costFunction(betaTest, layer, X[0:5000][None,:].flatten(), 5000, y, yUnique, iLambda = 0.)

(5000, 400)
(1, 400)


0.28762916516131876

## 1.1.3 Cost-Function Test
The outputs of the costFunction should be as follows:<br\>
betaTest, X, iLambda=0. — 0.287629 (Andrew Ng)<br\>
betaTest, X, iLambda=1. — 0.383770 (Andrew Ng)<br\>
betaTest, X, iLambda=0. — 0.0345203898838<br\>
betaInitial, X, iLambda=1. — 65.5961451562

In [8]:
print(costFunction(betaTest, layer, flatX, sampleSize, y, yUnique, iLambda = 0.))
print(costFunction(betaTest, layer, flatX, sampleSize, y, yUnique, iLambda = 1.))
print(costFunction(betaTest, layer, X[0][None,:].flatten(), 1, y, yUnique, iLambda = 0.))
print(costFunction(betaInitial, layer, flatX, sampleSize, y, yUnique, iLambda = 1.))

0.287629165161
0.384487796243
0.0345203898838
65.5961451562


## 1.2 Back Propagation
$\delta^l = H^l - Y$<br>
$\delta^{l-1} = (\beta^{l-1})^T\delta^l\cdot g'(h^{l-1})$

In [9]:
def backPropagation(flatBeta, layer, flatX, sampleSize, y, yUnique):
    Y = np.array([yUnique]* y.shape[0]) == y
    betaSet = reshapeBeta(flatBeta, layer)

    deltaSet = ()
#     hypothesis matrix E(5000, 10)
    H = forwardPropagation(flatBeta, layer, flatX, sampleSize)
#     error matrix E(5000, 10)
    E = H[len(layer)-2] - Y
    for l in reversed(range(len(layer)-1)):
        E = np.dot(E*sigmoidGradient(H[l]), betaSet[l])[:,1:]
        deltaSet = (np.dot(H[l].T, np.insert(E, 0, 1, axis=1)),) + deltaSet
    flatDelta = flattenBeta(deltaSet)
    return flatBeta + flatDelta/sampleSize

In [35]:
Y = np.array([yUnique]* y.shape[0]) == y
# print(Y.shape)
betaSet = reshapeBeta(betaTest, layer)
# print(len(betaSet))
deltaSet = ()
#     hypothesis matrix E(5000, 10)
H = forwardPropagation(betaTest, layer, flatX, sampleSize)
# print (len(H))
#     error matrix E(5000, 10)
E = H[len(layer)-2] - Y
# print(E.shape)
for l in reversed(range(len(layer)-1)):
    E = np.dot(E*sigmoidGradient(H[l]), betaSet[l])[:,1:]
    print(E.shape)
    deltaSet = (np.dot(H[l].T, np.insert(E, 0, 1, axis=1)),) + deltaSet
print(len(deltaSet))
print(deltaSet[0].shape)
print(deltaSet[1].shape)
flatDelta = flattenBeta(deltaSet)
print(betaTest.shape)
f = betaTest + flatDelta/sampleSize
f[3915]

(5000, 25)
(5000, 400)
2
(25, 401)
(10, 26)
(10285,)


0.14300247142022265

In [26]:
betaInitial = flattenBeta(generateBeta(layer))
a = backPropagation(betaTest, layer, flatX, sampleSize, y, yUnique)
print(a.shape)
print(a[3915])
print(np.sum(a))
print(costFunction(a,layer, flatX, sampleSize, y, yUnique, iLambda = 0.))

(10285,)
0.14300247142
-77.6897929714
0.328301599213


In [25]:
def gradientCheck(flatBeta, layer, flatX, sampleSize, y, yUnique, epsilon):
    for i in np.random.randint(flatBeta.size, size=10):
        epsilonVector = np.zeros(flatBeta.size)
        epsilonVector[i] = epsilon
        
        gradient = backPropagation(flatBeta, layer, flatX, sampleSize, y, yUnique)
        
        betaPlus = betaMinus = flatBeta
#         betaPlus = beta + epsilonVector
        betaPlus += epsilonVector
        costPlus = costFunction(betaPlus,layer, X, sampleSize, y, yUnique, iLambda = 0.)
#         betaMinus = beta - epsilonVector
        betaMinus -= epsilonVector
        costMinus = costFunction(betaMinus,layer, X, sampleSize, y, yUnique, iLambda = 0.)
        approximateGradient = (costPlus-costMinus)/(2*epsilon)
        print (i, '\t', approximateGradient, '\t', gradient[i])

epsilon = 0.0001
gradientCheck(betaTest, layer, flatX, sampleSize, y, yUnique, epsilon)


3915 	 -5.16694076413e-05 	 0.14300247142
2553 	 0.000130317904801 	 -0.393868746806
7889 	 -8.70829020005e-06 	 -0.00242495412
2259 	 -6.83055711992e-05 	 0.144925981932
3342 	 5.38021305285e-05 	 -0.178667553118
5405 	 -9.52503684259e-05 	 0.206407475658
5084 	 -4.85511891801e-05 	 0.172304406568
859 	 1.37250885657e-05 	 -0.0466439352383
3619 	 8.66365312824e-08 	 -0.000303325624571
5647 	 1.31436750372e-06 	 -0.0043613719981


http://www.holehouse.org/mlclass/09_Neural_Networks_Learning.html

In [ ]:
def betaOptimisation_1(flatBeta, flatX, sampleSize, y, yUnique, iLambda=0.):

    optimisedBeta = optimize.minimize(costFunction, flatBeta, args=(layer, flatX, sampleSize, y, yUnique),
                                      method=None, jac=backPropagation, options={'maxiter':50})

#     optimisedBeta = optimize.fmin_cg(costFunction, fprime=backPropagation, x0=flatBeta,
#                                      args=(layer, flatX, sampleSize, y, yUnique),
#                                      maxiter=50,disp=True,full_output=True)
    return(optimisedBeta)

In [ ]:
def betaOptimisation_2(flatBeta, flatX, sampleSize, y, yUnique, iLambda=0.):

#     optimisedBeta = optimize.minimize(costFunction, flatBeta, args=(layer, flatX, sampleSize, y, yUnique),
#                                       method=None, jac=backPropagation, options={'maxiter':50})

    optimisedBeta = optimize.fmin_cg(costFunction, fprime=backPropagation, x0=flatBeta,
                                     args=(layer, flatX, sampleSize, y, yUnique),
                                     maxiter=50,disp=True,full_output=True)
    return(optimisedBeta)

In [ ]:
a = betaOptimisation_1(betaInitial, flatX, sampleSize, y, yUnique, iLambda=0.)

In [ ]:
b = betaOptimisation_2(betaInitial, flatX, sampleSize, y, yUnique, iLambda=0.)

In [ ]:
def qualityControl(optimisedBeta, layer, flatX, sampleSize, y, yUnique, iLambda = 0.):
    X = flatX.reshape(sampleSize,-1)
    yAssignmentVector = []
    misAssignedIndex = []
    for n in range(sampleSize):
        x = X[n]
        yAssignment =  np.argmax(forwardPropagation(optimisedBeta, layer, X[n], 1)[1]) + 1
        if yAssignment == y[n]:
            yAssignmentVector += [True]
        else:
            yAssignmentVector += [False]
            misAssignedIndex += [n]
    return (sum(yAssignmentVector)/sampleSize)

In [ ]:
# neuralNetworkClassifier(, flatX, sampleSize, y, yUnique, iLambda=0.)
qualityControl(a['x'], layer, flatX, sampleSize, y, yUnique, iLambda = 0.)

In [ ]:
qualityControl(b[0], layer, flatX, sampleSize, y, yUnique, iLambda = 0.)